In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bert-base-uncased/vocab.txt
/kaggle/input/bert-base-uncased/config.json
/kaggle/input/bert-base-uncased/pytorch_model.bin
/kaggle/input/bert-base-multilingual-uncased/vocab.txt
/kaggle/input/bert-base-multilingual-uncased/config.json
/kaggle/input/bert-base-multilingual-uncased/pytorch_model.bin
/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv
/kaggle/input/hotel-reviews/7282_1.csv
/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews.csv
/kaggle/input/avishek-hotel/sentiments.csv


In [2]:
!pip install -q -U watermark

In [3]:
!pip install -qq transformers

In [4]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

CPython 3.7.6
IPython 7.12.0

numpy 1.18.1
pandas 1.0.1
torch 1.4.0
transformers 2.8.0


In [5]:
#@title Setup & Config
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
df=pd.read_csv("/kaggle/input/avishek-hotel/sentiments.csv",encoding='ISO-8859–1',header=0)
df.head()

,text,review
0,The premises are very peaceful and well mainta...,1
1,5 min drive out of Pisa or you need to catch t...,0
2,My family of 5 stayed at the residence San Ros...,1
3,We stayed here for a couple of nights on the w...,1
4,The accomodation was simple but more than adeq...,1


In [7]:
df.loc[ df['review'] ==1, 'review'] = 2

In [8]:
df

,text,review
0,The premises are very peaceful and well mainta...,2
1,5 min drive out of Pisa or you need to catch t...,0
2,My family of 5 stayed at the residence San Ros...,2
3,We stayed here for a couple of nights on the w...,2
4,The accomodation was simple but more than adeq...,2
...,...,...
58607,It is hard for me to review an oceanfront hote...,0
58608,"I live close by, and needed to stay somewhere ...",2
58609,Rolled in 11:30 laid out heads down woke up to...,2
58610,Absolutely terrible..I was told I was being gi...,-1


In [9]:
df.loc[ df['review'] ==0, 'review'] = 1

In [10]:
df

,text,review
0,The premises are very peaceful and well mainta...,2
1,5 min drive out of Pisa or you need to catch t...,1
2,My family of 5 stayed at the residence San Ros...,2
3,We stayed here for a couple of nights on the w...,2
4,The accomodation was simple but more than adeq...,2
...,...,...
58607,It is hard for me to review an oceanfront hote...,1
58608,"I live close by, and needed to stay somewhere ...",2
58609,Rolled in 11:30 laid out heads down woke up to...,2
58610,Absolutely terrible..I was told I was being gi...,-1


In [11]:
df.loc[ df['review'] ==-1, 'review'] = 0

In [12]:
df

,text,review
0,The premises are very peaceful and well mainta...,2
1,5 min drive out of Pisa or you need to catch t...,1
2,My family of 5 stayed at the residence San Ros...,2
3,We stayed here for a couple of nights on the w...,2
4,The accomodation was simple but more than adeq...,2
...,...,...
58607,It is hard for me to review an oceanfront hote...,1
58608,"I live close by, and needed to stay somewhere ...",2
58609,Rolled in 11:30 laid out heads down woke up to...,2
58610,Absolutely terrible..I was told I was being gi...,0


In [13]:
PRE_TRAINED_MODEL_NAME='bert-base-uncased'

In [14]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [15]:
sample_txt = 'When was I last outside? I am stuck at home for 2 weeks.'

In [16]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: When was I last outside? I am stuck at home for 2 weeks.
   Tokens: ['when', 'was', 'i', 'last', 'outside', '?', 'i', 'am', 'stuck', 'at', 'home', 'for', '2', 'weeks', '.']
Token IDs: [2043, 2001, 1045, 2197, 2648, 1029, 1045, 2572, 5881, 2012, 2188, 2005, 1016, 3134, 1012]


In [17]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [18]:
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

32


tensor([ 101, 2043, 2001, 1045, 2197, 2648, 1029, 1045, 2572, 5881, 2012, 2188,
        2005, 1016, 3134, 1012,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])

In [19]:
print(len(encoding['attention_mask'][0]))
encoding['attention_mask']

32


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [20]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

['[CLS]',
 'when',
 'was',
 'i',
 'last',
 'outside',
 '?',
 'i',
 'am',
 'stuck',
 'at',
 'home',
 'for',
 '2',
 'weeks',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [21]:
MAX_LEN = 160

In [22]:
class hotelDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [23]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [24]:
df_train, df_val, df_test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.9*len(df))])

In [25]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = hotelDataset(
    reviews=df.text.to_numpy(),
    targets=df.review.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [26]:
BATCH_SIZE = 16

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [27]:

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [28]:

last_hidden_state, pooled_output = bert_model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask']
)

In [29]:

class_names = ['negative', 'neutral', 'positive']

In [30]:
model = SentimentClassifier(len(class_names))
model = model.to(device)

In [31]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['text', 'input_ids', 'attention_mask', 'targets'])

In [32]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([16, 160])
torch.Size([16, 160])


In [33]:
F.softmax(model(input_ids, attention_mask), dim=1)

tensor([[0.1451, 0.2467, 0.6081],
        [0.2015, 0.3026, 0.4959],
        [0.3373, 0.3094, 0.3533],
        [0.1617, 0.3605, 0.4779],
        [0.3646, 0.3088, 0.3266],
        [0.2970, 0.3544, 0.3486],
        [0.3335, 0.3732, 0.2933],
        [0.2931, 0.3442, 0.3626],
        [0.1489, 0.2902, 0.5609],
        [0.1865, 0.3934, 0.4201],
        [0.2779, 0.2663, 0.4557],
        [0.3161, 0.2246, 0.4593],
        [0.2154, 0.2311, 0.5535],
        [0.2511, 0.3633, 0.3857],
        [0.2935, 0.2359, 0.4706],
        [0.1841, 0.3389, 0.4770]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [34]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [35]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [36]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [37]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/10
----------
Train loss 0.35628753007329694 accuracy 0.8594862045432388
Val   loss 0.3072123184068304 accuracy 0.8752772564408805

Epoch 2/10
----------
Train loss 0.2591433536500959 accuracy 0.9039436482402262
Val   loss 0.3476088004336676 accuracy 0.8700733663197407

Epoch 3/10
----------
Train loss 0.1907404961903193 accuracy 0.9416983523447402
Val   loss 0.5089410609561864 accuracy 0.8641869988056646

Epoch 4/10
----------
Train loss 0.14364883634779188 accuracy 0.9619528127132689
Val   loss 0.6062298182748757 accuracy 0.8638457601091964

Epoch 5/10
----------
Train loss 0.10377751486575627 accuracy 0.9750170615189626
Val   loss 0.7145956111373062 accuracy 0.8633339020644942

Epoch 6/10
----------
Train loss 0.07718525660926835 accuracy 0.9822072730817978
Val   loss 0.8239542720852087 accuracy 0.8640163794574305

Epoch 7/10
----------
Train loss 0.05579844426225733 accuracy 0.9875938383542946
Val   loss 0.873266039357748 accuracy 0.8635898310868453

Epoch 8/10
----------
T

In [38]:

test_acc, _ = eval_model(
  model,
  test_data_loader,
  loss_fn,
  device,
  len(df_test)
)

test_acc.item()

0.8623336745138179

In [39]:
def get_predictions(model, data_loader):
  model = model.eval()
  
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:

      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["review"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      probs = F.softmax(outputs, dim=1)

      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(probs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [40]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

KeyError: 'review'

In [41]:

print(classification_report(y_test, y_pred, target_names=class_names))

NameError: name 'y_test' is not defined